In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import pingouin as pg
import ppscore as pps
import statsmodels.api as sm
import matplotlib.pyplot as plt
import feature_engine
import pygwalker as pyg
from datetime import timedelta
from datetime import datetime
from sklearn.impute import SimpleImputer
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import sklearn
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from feature_engine.imputation import MeanMedianImputer
import math
import joblib
import pickle
import ipywidgets
from IPython.display import display


In [12]:
import joblib

# Load the occupancy model
occupancy_model = joblib.load('occupancy_model.pkl')

# Load the F&B revenue model
fandb_revenue_model = joblib.load('f&b_revenue_model.pkl') 

# Load the expenses model
expenses_model = joblib.load('expenses_model.pkl')

# # Load the linear regression models
# rooms_linear_regression_model = joblib.load('linear_regression_rooms_model.pkl')
# fandb_linear_regression_model = joblib.load('linear_regression_fandb_model.pkl')

# Prepare input data for the occupancy model
input_data_occupancy = {
    'Marketing': 500,
    'Seasonality': 2,
    'Average Room Rate': 80,
    # Add other relevant features for occupancy prediction
}

# Predict occupancy
input_array_occupancy = np.array([list(input_data_occupancy.values())])

# Predict occupancy
predicted_occupancy = occupancy_model.predict(input_array_occupancy)

# Calculate room revenue based on occupancy
number_of_rooms = 9
number_of_days = 30
room_rate = 80
room_revenue = predicted_occupancy * number_of_rooms * number_of_days * room_rate

# Prepare input data for the F&B revenue model
input_data_fandb_revenue = {
    'Rooms Revenue': room_revenue,
    'Percentage F&B Occ %': 0.4 ,  # Forecasted value
    'Percentage Rooms Occ %': predicted_occupancy,  # Assuming same as 'Percentage Rooms Occ %'
    # Add other relevant features for F&B revenue prediction
}
# Convert the input data dictionary into a numerical array
input_array_fandb_revenue = np.array([list(input_data_fandb_revenue.values())])

# Predict F&B revenue
predicted_fandb_revenue = fandb_revenue_model.predict(input_array_fandb_revenue)

# Calculate 'Total Revenue' as the sum of 'Rooms Revenue' and 'F&B Revenue'
total_revenue = room_revenue + predicted_fandb_revenue

# Prepare input data for the expenses model
input_data_expenses = {
    'Total Revenue': total_revenue,
    'Total Wages': 8000,
    'Insurances': 200,  
    'Transport': 300,
    'Marketing': 500,
    # Add other relevant features for expenses prediction
}

# Predict expenses
# Convert the input data dictionary into a numerical array
input_array_expenses = np.array([list(input_data_expenses.values())])

# Predict expenses
predicted_expenses = expenses_model.predict(input_array_expenses)

# Predict 'Rooms Expenses' based on 'Rooms Revenue'
input_data_rooms_expenses = {
    'Rooms Revenue': room_revenue,
    # Add other relevant features for rooms expenses prediction
}

# Convert the input data dictionary into a DataFrame
input_df_rooms_expenses = pd.DataFrame(input_data_rooms_expenses)

# Predict 'Rooms Expenses'
predicted_rooms_expenses = rooms_linear_regression_model.predict(input_df_rooms_expenses)

# Predict 'F&B Expenses' based on 'F&B Revenue'
input_data_fandb_expenses = {
    'F&B Revenue': predicted_fandb_revenue,
    # Add other relevant features for F&B expenses prediction
}

# Convert the input data dictionary into a DataFrame
input_df_fandb_expenses = pd.DataFrame(input_data_fandb_expenses)

# Predict 'F&B Expenses'
predicted_fandb_expenses = fandb_linear_regression_model.predict(input_df_fandb_expenses)

# Continue with the rest of the code as previously provided


# Convert NumPy arrays to Python floats
predicted_occupancy_float = predicted_occupancy.item()
room_revenue_float = room_revenue.item()
predicted_fandb_revenue_float = predicted_fandb_revenue.item()
total_revenue_float = total_revenue.item()
predicted_rooms_expenses_float = predicted_rooms_expenses.item()
predicted_fandb_expenses_float = predicted_fandb_expenses.item()
predicted_expenses_float = predicted_expenses.item()

# Print the formatted values
print("Predicted Occupancy: {:.2%}".format(predicted_occupancy_float))
print("Room Revenue: ${:.2f}".format(room_revenue_float))
print("Predicted F&B Revenue: ${:.2f}".format(predicted_fandb_revenue_float))
print("Total Revenue: ${:.2f}".format(total_revenue_float))
print("Predicted Rooms Expenses: ${:.2f}".format(predicted_rooms_expenses_float))
print("Predicted F&B Expenses: ${:.2f}".format(predicted_fandb_expenses_float))
print("Predicted Expenses: ${:.2f}".format(predicted_expenses_float))


# Calculate Gross Operating Profit (GOP)
gop = total_revenue_float - (predicted_rooms_expenses_float + predicted_fandb_expenses_float + predicted_expenses_float)

# Print the GOP
print("Gross Operating Profit (GOP): ${:.2f}".format(gop))


/Users/danielporras/hotel/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 3 features, but StandardScaler is expecting 5 features as input.